## Load functions and imports

In [33]:
import pandas as pd
from sklearn import datasets, linear_model
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import timeit
from sklearn.svm import SVR


def get_csv(filename,pred_y):
    #print(pred_y.shape)
    duration_content = np.rint(pred_y)
    #duration_content = np.array(duration_content)
    #print(duration_content.shape)
    
    #print(duration_content)
    id_content = np.array(test_id)
    #print(id_content.shape)

    df = pd.DataFrame({
        'id':id_content,
        'trip_duration':duration_content
    })

    # Create the csv file
    fn = filename + '.csv'
    df.to_csv(fn,index= False)
    
    
def get_csv_cluster(filename,pred_y,testid):
    #print(pred_y.shape)
    duration_content = np.rint(pred_y)
    #duration_content = np.array(duration_content)
    #print(duration_content.shape)
    
    #print(duration_content)
    id_content = np.array(testid)
    #print(id_content.shape)

    df = pd.DataFrame({
        'id':id_content,
        'trip_duration':duration_content
    })

    # Create the csv file
    fn = filename + '.csv'
    df.to_csv(fn,index= False)
    
    
def get_poly(degree,x,test):
    poly = PolynomialFeatures(degree=degree)
    x_ = poly.fit_transform(x)
    test_ = poly.fit_transform(test)
    
    return x_,test_

## Getting X, Y (Modify filename to get different version of data)

In [34]:
test = pd.read_csv('test.csv')

train_x = pd.read_csv('L2_pca_train_x.csv')   # Change this
train_y = pd.read_csv('train_y.csv',header = -1)   # Change this
test_x = pd.read_csv('L2_pca_test.csv')  # Change this
test_id = test['id']


In [35]:
print(np.array(train_x).shape)

(1458644, 5)


# Various regressions

### Linear Regression
0.869 1159 <br>
fit time is: 0.5949127580024651<br>
predict time is: 0.01478029000281822<br>

In [3]:
regr = linear_model.LinearRegression()


# Actual training
start_time = timeit.default_timer()
regr.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y = regr.predict(test_x)
pred_y = pred_y.flatten()

predict_elapsed = timeit.default_timer() - start_time

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_LR',pred_y)

/Users/wayne/anaconda3/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


fit time is: 0.3214782440045383
predict time is: 0.008671407995279878


### Polynomial Regression
0.84502 1151 <br>
fit time is: 12.516197695003939 <br>
predict time is: 0.01478029000281822 <br>

In [4]:

x_,test_ = get_poly(2,train_x,test_x)

regr = linear_model.LinearRegression()

# Actual training
start_time = timeit.default_timer()
regr.fit(x_, train_y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = regr.predict(test_)
pred_y = pred_y.flatten()
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_PR',pred_y)





fit time is: 1.757301732999622
predict time is: 0.008671407995279878


### Lasso

0.87012 1161<br>
fit time is: 0.6580201340038911 <br>
predict time is: 0.01415189600083977<br>

In [5]:


#clf = linear_model.Lasso(alpha=cv_list[max_idx])
clf = linear_model.Lasso(alpha=1.0)
# Actual training
start_time = timeit.default_timer()
clf.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()
pred_y = clf.predict(test_x)
pred_y = pred_y.flatten()
predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_LASSO',pred_y)

fit time is: 0.2738622929973644
predict time is: 0.010913894002442248


### Ridge
0.86982 1159
fit time is: 0.3165850510049495 <br>
predict time is: 0.010517886999878101 <br>

In [6]:

clf = Ridge(alpha=1.0)

# Actual training
start_time = timeit.default_timer()

clf.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()
pred_y = clf.predict(test_x)
pred_y = pred_y.flatten()
predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_Ridge',pred_y)

fit time is: 0.12245846599398647
predict time is: 0.009309730005043093


### Neural Network
0.88073  <br>
fit time is: 38.12283402299363 <br>
predict time is: 1.2900892430043314 <br>

In [31]:

clf = MLPRegressor(hidden_layer_sizes=(100,),  activation='relu', solver='adam',    alpha=0.001,batch_size='auto',
               learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
               random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
               nesterovs_momentum=True, early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
               epsilon=1e-08)


# Actual training
start_time = timeit.default_timer()
train_y= np.array(train_y).flatten()
clf.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()
pred_y = clf.predict(test_x)
#pred_y = pred_y.flatten()
predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_NN',pred_y)

fit time is: 28.407978393996018
predict time is: 0.7203619139909279


### Linear SVR

### Decision Tree Regression
0.61829 <br>
fit time is: 24.759334297996247 <br>
predict time is: 0.5807069809961831 <br>

In [8]:

clf = DecisionTreeRegressor()

# Actual training
start_time = timeit.default_timer()
train_y= np.array(train_y).flatten()
clf.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()
pred_y = clf.predict(test_x)
#pred_y = pred_y.flatten()
predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_DTR',pred_y)

fit time is: 27.384218589999364
predict time is: 0.3966126009981963


## EN
0.8891
fit time is: 0.673529542000324<BR>
predict time is: 0.0554104409966385<BR>

In [9]:

clf = ElasticNet(random_state=0)

# Actual training
start_time = timeit.default_timer()
train_y= np.array(train_y).flatten()
clf.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()
pred_y = clf.predict(test_x)
#pred_y = pred_y.flatten()
predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_ENet',pred_y)

fit time is: 0.227267189999111
predict time is: 0.006862378999358043


## RF Regression
0.82194<BR>
fit time is: 14.449963496997952<BR>
predict time is: 0.2338456350043998<BR>

In [10]:
clf = RandomForestRegressor(max_depth=2, random_state=0)

# Actual training
start_time = timeit.default_timer()
train_y= np.array(train_y).flatten()
clf.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()
pred_y = clf.predict(test_x)
#pred_y = pred_y.flatten()
predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_RFR',pred_y)

fit time is: 19.497659218999615
predict time is: 0.1329097779962467


## XGBR

In [22]:

clf = XGBRegressor()

# Actual training
start_time = timeit.default_timer()
train_y= np.array(train_y).flatten()
clf.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()
pred_y = clf.predict(test_x)
#pred_y = pred_y.flatten()
predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SUB_PCA_L2_XGB',pred_y)

fit time is: 109.12713578800322
predict time is: 1.2864535200060345


## Cluster Transformation

### Find best k for the cluster

In [11]:
num_list = [2,3,4,5,6]
size_list = [10000]
results = []
for n_cluster in num_list:
    kmeans = KMeans(n_clusters=n_cluster).fit(train_x)
    label = kmeans.fit_predict(train_x)
    for size in size_list:
        sil_coeff = silhouette_score(train_x[:size], label[:size], metric='euclidean')
        results.append(sil_coeff)
        #store.append(sil_coeff)
        print("For n_clusters={}, The Silhouette Coefficient with a size of {} is {} ".format(n_cluster, size,sil_coeff))
    

For n_clusters=2, The Silhouette Coefficient with a size of 10000 is 0.6064859745655558 
For n_clusters=3, The Silhouette Coefficient with a size of 10000 is 0.5841746957777221 
For n_clusters=4, The Silhouette Coefficient with a size of 10000 is 0.5847585564926365 
For n_clusters=5, The Silhouette Coefficient with a size of 10000 is 0.5765903192835452 
For n_clusters=6, The Silhouette Coefficient with a size of 10000 is 0.5682497055407519 


## Split training set based on cluster

In [12]:
best_k = num_list[np.argmax(np.array(results))]
print(best_k)

kmeans = KMeans(n_clusters=best_k).fit(train_x)
labels = kmeans.predict(train_x)
#print(labels)

train_x_cluster = train_x.copy()

train_x_cluster['cluster'] = labels
train_x_cluster['duration'] = train_y

mask = train_x_cluster['cluster'] ==0
train_x_c0 = train_x_cluster[mask]
train_x_c1 = train_x_cluster[~mask]


train_y_c0 = train_x_c0['duration']
train_y_c1 = train_x_c1['duration']
train_x_c0 = train_x_c0.drop(['duration'],axis=1)
train_x_c1 = train_x_c1.drop(['duration'],axis=1)

predict_labels = kmeans.predict(test_x)
test_x_temp = test_x.copy()
test_x_temp['cluster'] = predict_labels
test_x_temp['testid'] = test_id



mask = test_x_temp['cluster'] ==0
test_x_c0 = test_x_temp[mask]
test_x_c1 = test_x_temp[~mask]

test_id_c0 = test_x_c0['testid']
test_x_c0 = test_x_c0.drop(['testid'],axis=1)
test_id_c1 = test_x_c1['testid']
test_x_c1 = test_x_c1.drop(['testid'],axis=1)

2


## Re-Train Linear
From 0.869  to 0.86137

In [13]:

regr1 = linear_model.LinearRegression()
regr2 = linear_model.LinearRegression()

# Actual training
start_time = timeit.default_timer()
regr1.fit(train_x_c0, train_y_c0)
regr2.fit(train_x_c1, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_x_c0)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test_x_c1)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_LR_Cluster',pred,testid)

fit time is: 0.26683465899986913
predict time is: 0.02153128200006904


## Re-Train PR
From 0.84502 to 0.81918

In [14]:
x_,test_ = get_poly(2,train_x_c0,test_x_c0)
x2_,test2_ = get_poly(2,train_x_c1,test_x_c1)


regr1 = linear_model.LinearRegression()
regr2 = linear_model.LinearRegression()

# Actual training
start_time = timeit.default_timer()
regr1.fit(x_, train_y_c0)
regr2.fit(x2_, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test2_)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_PR_Cluster',pred,testid)

fit time is: 2.230023687996436
predict time is: 0.018422982000629418


## ReTrain Lasso
0.87012 to 0.86344

In [15]:

regr1 = linear_model.Lasso(alpha=1.0)
regr2 = linear_model.Lasso(alpha=1.0)

# Actual training
start_time = timeit.default_timer()
regr1.fit(train_x_c0, train_y_c0)
regr2.fit(train_x_c1, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_x_c0)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test_x_c1)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_LASSO_Cluster',pred,testid)

fit time is: 0.2760903300004429
predict time is: 0.024217270998633467


## ReTrain Ridge
From 0.86982 to 0.86138

In [16]:

regr1 = Ridge(alpha=1.0)
regr2 = Ridge(alpha=1.0)

# Actual training
start_time = timeit.default_timer()
regr1.fit(train_x_c0, train_y_c0)
regr2.fit(train_x_c1, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_x_c0)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test_x_c1)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_Ridge_Cluster',pred,testid)

fit time is: 0.16474085400113836
predict time is: 0.025547787998220883


## RT NN
From 0.88073 to 0.87623

In [32]:

regr1 =  MLPRegressor(hidden_layer_sizes=(100,),  activation='relu', solver='adam',    alpha=0.001,batch_size='auto',
               learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
               random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
               nesterovs_momentum=True, early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
               epsilon=1e-08)
regr2 =  MLPRegressor(hidden_layer_sizes=(100,),  activation='relu', solver='adam',    alpha=0.001,batch_size='auto',
               learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
               random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
               nesterovs_momentum=True, early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
               epsilon=1e-08)

# Actual training
start_time = timeit.default_timer()
regr1.fit(train_x_c0, train_y_c0)
regr2.fit(train_x_c1, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_x_c0)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test_x_c1)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_NN_Cluster',pred,testid)

fit time is: 43.08801542798756
predict time is: 0.6636643959936919


## RT DTR
From 0.61829 to 0.62575

In [18]:

regr1 = DecisionTreeRegressor()
regr2 = DecisionTreeRegressor()

# Actual training
start_time = timeit.default_timer()
regr1.fit(train_x_c0, train_y_c0)
regr2.fit(train_x_c1, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_x_c0)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test_x_c1)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_DTR_Cluster',pred,testid)

fit time is: 20.53099657400162
predict time is: 0.3283661210007267


## RT EN
0.88832

In [19]:

regr1 = ElasticNet(random_state=0)
regr2 = ElasticNet(random_state=0)

# Actual training
start_time = timeit.default_timer()
regr1.fit(train_x_c0, train_y_c0)
regr2.fit(train_x_c1, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_x_c0)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test_x_c1)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_ENet_Cluster',pred,testid)

fit time is: 0.31722642200475093
predict time is: 0.025038630999915767


## RT RF
0.82167<BR>
fit time is: 14.623066553998797 <BR>
predict time is: 0.13730818399926648<BR>

In [20]:
regr1 = RandomForestRegressor(max_depth=2, random_state=0)
regr2 = RandomForestRegressor(max_depth=2, random_state=0)

# Actual training
start_time = timeit.default_timer()
regr1.fit(train_x_c0, train_y_c0)
regr2.fit(train_x_c1, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_x_c0)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test_x_c1)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_RF_Cluster',pred,testid)

fit time is: 15.77582739799982
predict time is: 0.10597184699872741


## RT XGB

In [24]:
regr1 = XGBRegressor()
regr2 = XGBRegressor()

# Actual training
start_time = timeit.default_timer()
regr1.fit(train_x_c0, train_y_c0)
regr2.fit(train_x_c1, train_y_c1)
fit_elapsed = timeit.default_timer() - start_time

start_time = timeit.default_timer()

pred_y1 = regr1.predict(test_x_c0)
pred_y1 = pred_y1.flatten()

pred_y2 = regr2.predict(test_x_c1)
pred_y2 = pred_y2.flatten()

predict_elapsed = timeit.default_timer() - start_time

pred = np.append(pred_y1,pred_y2)
testid = np.append(test_id_c0,test_id_c1)

print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv_cluster('SUB_PCA_L2_XGB_Cluster',pred,testid)

fit time is: 91.56019852499594
predict time is: 1.2278710449973005
